In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

file_path = "../../data/processed/medical_equipment_utilization_synthetic_cleaned(in).csv"

raw_df = pd.read_csv(file_path)
raw_df.columns = [c.strip().replace(" ", "_") for c in raw_df.columns]

df = raw_df.dropna(axis=1, how="all").copy()

numeric_cols = [
    "Available_Hours",
    "Actual_Operating_Hours",
    "Utilization_Rate",
    "Number_of_Procedures",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

by_type = (
    df.groupby("Equipment_Type")
    .agg(
        avg_utilization_rate=("Utilization_Rate", "mean"),
        avg_procedures_per_day=("Number_of_Procedures", "mean"),
        avg_operating_hours=("Actual_Operating_Hours", "mean"),
    )
)

by_dept = (
    df.groupby("Department")
    .agg(
        avg_utilization_rate=("Utilization_Rate", "mean"),
        avg_procedures_per_day=("Number_of_Procedures", "mean"),
    )
)

if "Month" in df.columns:
    by_month = (
        df.groupby("Month")
        .agg(
            avg_utilization_rate=("Utilization_Rate", "mean"),
            total_procedures=("Number_of_Procedures", "sum"),
        )
        .sort_values("avg_utilization_rate", ascending=False)
    )
else:
    by_month = None

by_type.head()


In [ ]:
fig_util_type = px.bar(
    by_type.reset_index(),
    x="Equipment_Type",
    y="avg_utilization_rate",
    hover_data=["avg_operating_hours", "avg_procedures_per_day"],
    title="Average Utilization Rate by Equipment Type",
)
fig_util_type.update_layout(xaxis_tickangle=-45)
fig_util_type.show()


In [ ]:
fig_proc_type = px.bar(
    by_type.reset_index(),
    x="Equipment_Type",
    y="avg_procedures_per_day",
    title="Average Procedures per Day by Equipment Type",
)
fig_proc_type.update_layout(xaxis_tickangle=-45)
fig_proc_type.show()


In [ ]:
fig_util_dept = px.bar(
    by_dept.reset_index(),
    x="Department",
    y="avg_utilization_rate",
    title="Average Utilization Rate by Department",
)
fig_util_dept.update_layout(xaxis_tickangle=-30)
fig_util_dept.show()

if by_month is not None:
    fig_util_month = px.bar(
        by_month.reset_index(),
        x="Month",
        y="avg_utilization_rate",
        title="Average Utilization Rate by Month",
    )
    fig_util_month.update_layout(xaxis_tickangle=-30)
    fig_util_month.show()
